## Load useful libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from badassdatascience.forex.database.mongodb.pull_forex_data_from_database.pull_forex_data_from_database import query_single_instrument_and_single_granularity 
from badassdatascience.forex.EDA.BadassTimeSeries import BadassTimeSeries

## User settings

These are currently hard-coded for this effort, but should this analysis become part of a pipeline I'll convert these to runtime arguments.

In [3]:
output_directory = 'output'
instrument = 'EUR/USD'
granularity = 'H1' # 'M15'

## Load the data

This pulls data from either a recently saved file or from the source Mongo database.

In [4]:
save_file = output_directory + '/df_' + instrument.replace('/', '_') + '__' + granularity + '.pickled'

df = query_single_instrument_and_single_granularity(
    instrument = instrument,
    granularity = granularity,
    pull_from_database = False,
    save_file = save_file,
)

## Visual QA #1

In [5]:
df.head(2)

,_id,volume,time,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,time_iso_pd_datetime
0,68097c78da2b3e213f22f086,3,1262538000,EUR/USD,H1,2010-01-03T12:00:00-05:00,6,12,1.43070,1.43097,...,1.43247,0.00077,0.00077,1.43120,1.43172,1.43120,1.43172,0.00052,0.00052,2010-01-03 12:00:00-05:00
1,68097c78da2b3e213f22f087,137,1262541600,EUR/USD,H1,2010-01-03T13:00:00-05:00,6,13,1.43097,1.43325,...,1.43222,-0.00025,0.00348,1.43172,1.43425,1.43105,1.43157,-0.00015,0.00320,2010-01-03 13:00:00-05:00


In [6]:
df.tail(2)

,_id,volume,time,instrument,granularity,time_iso,weekday,hour,bid_o,bid_h,...,ask_c,ask_return,ask_volatility,mid_o,mid_h,mid_l,mid_c,mid_return,mid_volatility,time_iso_pd_datetime
97953,6863c0e142919cd524d22b01,7700,1751364000,EUR/USD,H1,2025-07-01T06:00:00-04:00,1,6,1.18131,1.18227,...,1.18124,-0.00021,0.00125,1.18138,1.18234,1.18110,1.18116,-0.00022,0.00124,2025-07-01 06:00:00-04:00
97954,6863cef18b3fe0e82e5f9162,7564,1751367600,EUR/USD,H1,2025-07-01T07:00:00-04:00,1,7,1.18106,1.18187,...,1.18147,0.00025,0.00136,1.18114,1.18194,1.18058,1.18140,0.00026,0.00136,2025-07-01 07:00:00-04:00


## Run additional configuration and time series processing

In [ ]:
ts = BadassTimeSeries(df['time'], df['mid_c'])
ts.fit()
ts.fit_details()

## Review stationarity test results

The "BadassTimeSeries" object contains four time series, each of which are tested for stationarity using both ADF and KPSS tests.

Augmented Dickey-Fuller (ADF) tests the null hypothesis that a time series has a unit root, while Kwiatkowski-Phillips-Schmidt-Shin (KPSS) tests whether the trend is stationary:

In [ ]:
ts.df_stationarity_test_table

## Use Discrete Fourier Transform to Identify Fundamental Frequencies

In [ ]:
def use_dft_to_find_fundamental_frequencies(x, n_nearby = 200, n_order = 10):
    fft_x = np.fft.fft(x)
    N = len(x)
    magnitudes = np.abs(fft_x)[0:(N // 2)]
    
    # we only care about frequency magnitudes within the reasonable length of the time series we will use to train the RNN
    nearby = magnitudes[0:n_nearby] 
    
    order = np.argsort(nearby)
    return order[(-1 * n_order):]

In [ ]:
frequencies_200 = use_dft_to_find_fundamental_frequencies(ts.df_diff_filled_forward['y'])
frequencies_300 = use_dft_to_find_fundamental_frequencies(ts.df_diff_filled_forward['y'], n_nearby = 300)

print(frequencies_200)
print(frequencies_300)

In [ ]:
ts.plot_the_pacfs()